# Non-Ergodic GMM Predictions

This notebook is used to make predictions for new scenarios and events for non-ergodic ground motion models developed with the provided tools.

This script was developed as part of the Non-ergodic Methodology and Modeling Tools research project summarized in the report by Lavrentiadis G., Kuehn N., Bozorgnia Y., Seylabi E., Meng X., Goulet C., and Kottke A. (2022), "Non‐ergodic Methodology and Modeling Tools (Report GIRS-2022-04)." Natural Hazards Risk and Resiliency Research Center, University of California, Los Angeles.

Support provided by the California Department of Transportation (Caltrans) and Pacific Gas and Electric Company (PG&E) is gratefully acknowledged. 

## Load Required Libraries 

In [ ]:
#load variables
import os
import sys
import pathlib
#arithmetic libraries
import numpy as np
#statistics libraries
import pandas as pd
#plot libraries
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import cartopy.crs as ccrs
import cartopy.feature as cfeature

#user functions
sys.path.insert(0,'../Python_lib/ground_motions')
import pylib_NGMM_prediction as pyngmm
sys.path.insert(0,'../Python_lib/plotting')
import pylib_contour_plots as pycplt

## Input

In [ ]:
#number of samples
n_samp = 10

### Regression Files
The following block specifies the paths for the outpput regression files.

In [ ]:
#hyper-parameters
fname_hyp    = '../../Data/Regression/example_ds2_inla/example_regression_ds2_inla_inla_hyperparameters.csv'
#non-ergodic coefficients
fname_coeffs = '../../Data/Regression/example_ds2_inla/example_regression_ds2_inla_inla_coefficients.csv'
#non-ergodic anelastic attenuation
fname_cellinfo  = '../../Data/Flatfiles/examp_datasets/CatalogNGAWest3CALite_cellinfo.csv'
fname_cellatten = '../../Data/Regression/example_ds2_inla/example_regression_ds2_inla_inla_catten.csv'

#read regression files
df_nerg_hyp       = pd.read_csv(fname_hyp,       index_col=0)
df_nerg_coeffs    = pd.read_csv(fname_coeffs,    index_col=0)
df_nerg_cellinfo  = pd.read_csv(fname_cellinfo,  index_col=0)
df_nerg_cellatten = pd.read_csv(fname_cellatten, index_col=0)

### Prediction Files

The next block specifies the path for the flatfile for the ground-motion predictions.

In [ ]:
#prediction scenarios
fname_scen = '../../Data/Prediction/scen_predict.csv'

#read prediction scenario
df_predict = pd.read_csv(fname_scen, index_col=0)

### Output File
Specifed the location of the output file for the generated scenarios

In [ ]:
fname_predict_out = '../../Data/Prediction/scen_nerg_predict.csv'

### Non-ergodic Functional Form
The `nerg_coeff_info` dictionary specifies the structure of the spatially varying coefficient component of the non-ergodic ground motion model. 
The `nerg_atten_info` dictionary specifies the structure of the cell-specific anelastic attenuation.

In [ ]:
# Non-ergodic Functional Form
# -----------------------
#non-ergodic coefficient information
nerg_coeff_info = {'dc0':   {'coeff': ['dc_0_mean','dc_0_sig'],     'scaling':None, 
                             'kernel_type':0},
                   'dc1e':  {'coeff': ['dc_1e_mean','dc_1e_sig'],   'scaling':None, 'cor_info':['eqX','eqY'],
                             'kernel_type':2, 
                             'hyp':{'ell':df_nerg_hyp.loc['mean','ell_1e'], 'omega':df_nerg_hyp.loc['mean','omega_1e'], 'nu:':1.5}},
                   'dc1as': {'coeff': ['dc_1as_mean','dc_1as_sig'], 'scaling':None, 'cor_info':['staX','staY'],
                             'kernel_type':2,
                             'hyp':{'ell':df_nerg_hyp.loc['mean','ell_1as'], 'omega':df_nerg_hyp.loc['mean','omega_1as'], 'nu:':1.5}},
                   'dc1bs': {'coeff': ['dc_1bs_mean','dc_1bs_sig'], 'scaling':None, 'cor_info':'ssn',
                             'kernel_type':1,
                             'hyp':{'omega':df_nerg_hyp.loc['mean','omega_1bs']}}
                   }

#cell-specific anelastic attenuation infromation
nerg_atten_info = {'cap':    {'catten': ['c_cap_mean','c_cap_sig'],  'cor_info':'cellid',
                              'kernel_type':1,
                              'hyp':{'mean_ca': df_nerg_hyp.loc['mean','mu_cap'], 'omega':df_nerg_hyp.loc['mean','omega_cap']}}
                   }

## Non-Ergodic Predictions

In [ ]:
ngmm_prdct_samp, _, _, ngmm_prdct_mu, ngmm_prdct_sig, _, _ = pyngmm.PredictNErgEffects(n_samp, nerg_coeff_info, df_predict,  df_nerg_coeffs, 
                                                                                       nerg_catten_info=nerg_atten_info, 
                                                                                       df_cell_info=df_nerg_cellinfo, 
                                                                                       df_nerg_cellatten=df_nerg_cellatten)

## Output

In [ ]:
#median ngmm
df_predict['nerg_mu']  = df_predict.erg_base.values * np.exp(ngmm_prdct_mu)
#epistemic uncertainty
df_predict['nerg_sig'] = ngmm_prdct_sig
#random realizations
cname = [f'nerg_rlz_%i'%k for k in range(n_samp)]
df_predict[cname] = df_predict.erg_base.values[:,np.newaxis] * np.exp(ngmm_prdct_samp)

df_predict.to_csv(fname_predict_out)

### Summary Plots

In [ ]:
#median ground motion
fname_fig = 'examp_eas_med'
cbar_label = r'$EAS_{nerg, med}$ (g sec)'
data2plot  = df_predict[['staLat','staLon','nerg_mu']].values
fig, ax, cbar, data_crs, _ = pycplt.PlotScatterCAMap(data2plot, cmap='RdYlBu_r', log_cbar=True, marker_size=60,
                                                     frmt_clb = '%.0E')
#update colorbar
cbar.ax.tick_params(tick1On=1, labelsize=30)
cbar.set_label(cbar_label, size=35)
#grid lines
gl = ax.gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right = False
gl.xlabel_style = {'size': 30}
gl.ylabel_style = {'size': 30}
gl.xlocator = mticker.FixedLocator([-123, -119, -115])
gl.ylocator = mticker.FixedLocator([ 33,   37,   41])
#add water bodies
oceans = cfeature.NaturalEarthFeature(category='physical', name='ocean',
                                      scale='50m')
ax.add_feature(oceans, zorder=50)
#apply tight layout
fig.tight_layout()

In [ ]:
#epistemic uncertainty
fname_fig = 'examp_eas_unc'
cbar_label = r'$\psi$'
data2plot  = df_predict[['staLat','staLon','nerg_sig']].values
fig, ax, cbar, data_crs, _ = pycplt.PlotScatterCAMap(data2plot, cmap='RdYlBu_r', log_cbar=False, marker_size=60,
                                                     frmt_clb = '%.2f')
#update colorbar
cbar.ax.tick_params(tick1On=1, labelsize=30)
cbar.set_label(cbar_label, size=35)
#grid lines
gl = ax.gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right = False
gl.xlabel_style = {'size': 30}
gl.ylabel_style = {'size': 30}
gl.xlocator = mticker.FixedLocator([-123, -119, -115])
gl.ylocator = mticker.FixedLocator([ 33,   37,   41])
#add water bodies
oceans = cfeature.NaturalEarthFeature(category='physical', name='ocean',
                                      scale='50m')
ax.add_feature(oceans, zorder=50)
#apply tight layout
fig.tight_layout()

In [ ]:
#random realization
fname_fig = 'examp_eas_realiz'
cbar_label = r'$EAS_{nerg, realiz}$ (g sec)'
data2plot  = df_predict[['staLat','staLon','nerg_rlz_1']].values
fig, ax, cbar, data_crs, _ = pycplt.PlotScatterCAMap(data2plot, cmap='RdYlBu_r', log_cbar=True, marker_size=60,
                                                     frmt_clb = '%.0E')
#update colorbar
cbar.ax.tick_params(tick1On=1, labelsize=30)
cbar.set_label(cbar_label, size=35)
#grid lines
gl = ax.gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right = False
gl.xlabel_style = {'size': 30}
gl.ylabel_style = {'size': 30}
gl.xlocator = mticker.FixedLocator([-123, -119, -115])
gl.ylocator = mticker.FixedLocator([ 33,   37,   41])
#add water bodies
oceans = cfeature.NaturalEarthFeature(category='physical', name='ocean',
                                      scale='50m')
ax.add_feature(oceans, zorder=50)
#apply tight layout
fig.tight_layout()